# Projeto Final - Detecção de Falhas em Chapas de Aço Inoxidável

Uma abordagem com aprendizado de máquina para controle de qualidade industrial.

## 1. Problema
Na indústria de chapas de aço, detectar falhas manualmente é custoso, demorado e sujeito a erros.O objetivo é desenvolver um modelo que classifique automaticamente o tipo de defeito em cada chapa com base em medidas geométricas, estatísticas e de luminosidade.

## 2. O Dataset
O conjunto de dados contém 31 colunas de características numéricas e 7 colunas indicando o tipo de falha (falha_1 até falha_outros). Cada linha representa uma chapa de aço.

Carregando os dados:

In [ ]:
import pandas as pd # type: ignore
df = pd.read_csv("Bootcamp_Train.csv")
df.head()

## 3. Limpeza e Pré-processamento
Corrigir valores inválidos e padronizar as variáveis.

In [ ]:
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.preprocessing import StandardScaler # type: ignore

falhas = ['falha_1', 'falha_2', 'falha_3', 'falha_4', 'falha_5', 'falha_6', 'falha_outros']
df['classe'] = df[falhas].idxmax(axis=1)
X = df.drop(columns=['id'] + falhas + ['classe'])
y = df['classe']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 4. Treinamento com LightGBM
Treinamos o modelo com os dados de treino e avaliamos com os dados de teste.

In [ ]:
import lightgbm as lgb # type: ignore
from sklearn.metrics import classification_report # type: ignore

modelo = lgb.LGBMClassifier(random_state=42)
modelo.fit(X_train_scaled, y_train)

y_pred = modelo.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

## 5. Visualização: Importância das Variáveis

In [ ]:
import matplotlib.pyplot as plt # type: ignore
lgb.plot_importance(modelo, max_num_features=10)
plt.show()

## 6. Interpretação com SHAP
Entendi como o modelo toma suas decisões.

In [ ]:
import shap # type: ignore
explainer = shap.Explainer(modelo)
shap_values = explainer(X_test_scaled)
shap.summary_plot(shap_values, X_test, plot_type='bar')

## 7. Conclusões 

- O modelo obteve um F1-score macro de aproximadamente 0.91.
- As variáveis mais importantes foram área, brilho, espessura e tipo do aço.
